In [7]:
# Editor: ThreeLiu
# Time: 2023/3/31
# Course: NLP
# Description: 分词实验，使用最大匹配算法分词，构建trie树

import transfer_code
import pandas as pd

In [8]:
train_path = 'train.csv/train.csv'
test_path = 'test.csv'

In [9]:
class TrieNode:
    def __init__(self, word: str):
        self.children = {}
        self.word = word

    def judge(self):
        if 'end' in self.children.keys():
            return True
        else:
            return False


def add_node(node: TrieNode, word: str):
    for char in word:
        new_node = TrieNode(word=char)
        node.children[char] = new_node
        node = new_node
    node.children['end'] = TrieNode(word='end')


class Trie:
    print_num = 0

    def __init__(self):
        self.root = TrieNode(word='root')

    def build_tree(self, word: str):
        if len(self.root.children) == 0:  # 如果根节点为空，直接添加
            add_node(word=word, node=self.root)
            return 'add'
        else:
            node = self.root
            for charnum in range(len(word)):
                if word[charnum] in node.children.keys():  # 如果当前字在节点中，继续往下找
                    node = node.children.get(word[charnum])
                    if charnum == len(word) - 1 and node.judge:  # 如果当前字是最后一个字，返回exist
                        return 'exist'
                    elif charnum == len(word) - 1 and not node.judge:  # 如果当前字是最后一个字，但是节点中没有end，添加end
                        node.children['end'] = TrieNode(word='end')
                        return 'add'
                    else:  # 如果当前字不在节点中，添加节点
                        continue
                else:
                    add_node(word=word[charnum:], node=node)
                    return 'add'

    def print(self, node=None):  # 打印trie树
        if node is None:
            node = self.root
        print(node.word, end='')
        if node.children:
            flag = False
            for child in node.children.values():
                if flag:
                    print((self.print_num + 1) * '\t', end='')
                flag = True
                self.print_num += 1
                print('->', end='')
                self.print(child)
        else:
            print(' len:', self.print_num - 1)

            # print(self.print_num * '----', end='')

            self.print_num -= 1
            return
        self.print_num -= 1

    def search(self, word):  # 查找单词是否在trie树中
        node = self.root
        for char in word:
            if char in node.children.keys():
                node = node.children.get(char)
            else:
                return False
        return True

# trie = Trie()
# text1=['我们','我们要','我们要去','我们要去吃','我们要去吃饭','我们要去玩','我们要去吃火锅']
# for word in text1:
#     trie.build_tree(word)
# trie.print()


In [10]:
# 分块读取训练集并分词构建trie树
import re
import sys

text_total = []
trie = Trie()
count = 0


def remove_chinese_punctuation(lst):
    pattern = re.compile(r'[\u4e00-\u9fa5%％\d]')
    # pattern = re.compile(r'[\u4e00-\u9fa5]')
    return [x for x in lst if pattern.search(x)]


for chunk in pd.read_csv(train_path, chunksize=1000):

    text = []
    for i in range(len(chunk)):
        text.extend(chunk['sentence'].tolist()[i].split())
        count += 1

    # text = remove_chinese_punctuation(text)
    for word in text:
        trie.build_tree(word)
    text_total.extend(text)

# trie.print()
print('counter:', count)
print('list_size:', sys.getsizeof(set(text_total)))
print('trie_size:', sys.getsizeof(trie))
print('len:', len(text_total))
print('unique_len:', len(set(text_total)))
# counter: 86918
# list_size: 4194520
# trie_size: 48
# len: 1998791
# unique_len: 87739
# print(text_total)
text_total = set(text_total)

counter: 86918
list_size: 4194520
trie_size: 48
len: 2368391
unique_len: 88119


In [12]:
# 使用text_total采用最大匹配算法分词
test_data = pd.read_csv(test_path)
data_list = test_data['sentence'].tolist()

# 正向最大匹配算法
def max_match(sen):
    sen = sen.strip()
    # sen = add_spaces_to_numbers(sen)
    sen = sen.split()
    result = []
    for word in sen:
        if word in text_total:
            result.append(word)
        else:
            left = 0
            right = len(word)
            while left < right:
                if trie.search(word[left:right]):
                    result.append(word[left:right])
                    right = left
                    left = 0
                else:
                    left += 1

    # return result  # 返回分词结果
    return result[::-1]  # 返回分词结果

res_dict = {}
for i in range(len(data_list)):
    sen = data_list[i]
    # sen = remove_chinese_punctuation(sen)
    print(''.join(sen))
    sen = max_match(''.join(sen))
    # 将列表转换为字符串，每个元素之间用空格隔开
    sen = ' '.join(sen)
    print(sen)
    result = transfer_code.transfer(sen)
    # print(result)
    res_dict[i] = result

import csv

with open('result.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'expected'])
    for key, value in res_dict.items():
        writer.writerow([key + 1, value])

扬帆远东做与中国合作的先行
扬帆 远东 做 与 中国 合作 的 先行
希腊的经济结构较特殊。
希腊 的 经济 结构 较 特殊 。
海运业雄踞全球之首，按吨位计占世界总数的１７％。
海运 业 雄踞 全球 之 首 ， 按 吨位 计 占 世界 总数 的 １７％ 。
另外旅游、侨汇也是经济收入的重要组成部分，制造业规模相对较小。
另外 旅游 、 侨汇 也是 经济 收入 的 重要 组成部分 ， 制造业 规模 相对 较小 。
多年来，中希贸易始终处于较低的水平，希腊几乎没有在中国投资。
多年来 ， 中 希 贸易 始终 处于 较低 的 水平 ， 希腊 几乎 没有 在 中国 投资 。
十几年来，改革开放的中国经济高速发展，远东在崛起。
十 几年来 ， 改革开放 的 中国经济 高速 发展 ， 远东 在 崛起 。
瓦西里斯的船只中有４０％驶向远东，每个月几乎都有两三条船停靠中国港口。
瓦西 里斯 的 船只 中 有 ４０％ 驶 向 远东 ， 每个 月 几乎 都 有 两 三条 船 停靠 中国 港口 。
他感受到了中国经济发展的大潮。
他 感 受到 了 中国经济 发展 的 大潮 。
他要与中国人合作。
他 要 与 中国人 合作 。
他来到中国，成为第一个访华的大船主。
他 来到 中国 ， 成为 第一个 访 华 的 大船 主 。
访问归来，他对中国发展充满信心，他向希腊海运部长介绍了情况，提出了两国在海运、造船业方面合作的建议。
访问 归来 ， 他 对 中国 发展 充满 信心 ， 他 向 希腊 海运 部长 介绍 了 情况 ， 提出 了 两国 在 海运 、 造船 业 方面 合作 的 建议 。
１９９５年１０月，希腊海运部长访华时，他根据“船长”的建议与中方探讨了在海运、造船方面合作的可能与途径。
１９９５年１０月 ， 希腊 海运 部长 访 华 时 ， 他 根据 “ 船长 ” 的 建议 与 中方 探讨 了 在 海运 、 造船 方面 合作 的 可能 与 途径 。
“船长”本人还与几个船主联合起来准备与我远洋公司建立合资企业。
“ 船长 ” 本人 还 与 几个 船 主 联合 起来 准备 与 我 远洋 公司 建立 合资 企业 。
“船长”常说，要么不干，干就要争第一。
“ 船长 ” 常 说 ， 要么 不干 ， 干 就要 争 第一 。
他拥有世界最大的私人集装箱船队，也要做与中国合作的先行。
他 拥有 世

In [13]:
# 使用trie树采用最大匹配算法分词
test_data = pd.read_csv(test_path)
data_list = test_data['sentence'].tolist()


# print(data_list)
def add_spaces_to_numbers(sen):
    pattern = re.compile(r'\d+％')
    matches = pattern.findall(sen)
    for match in matches:
        sen = sen.replace(match, f' {match} ')
    return sen


# # 双向最大匹配算法
# def bi_max_match(sen):
#     sen = sen.strip()
#     sen = add_spaces_to_numbers(sen)
#     sen = sen.split()
#     result = []
#     for word in sen:
#         if trie.search(word):
#             result.append(word)
#         else:
#             left = 0
#             right = len(word)
#             while left < right:
#                 if trie.search(word[left:right]):
#                     result.append(word[left:right])
#                     left = right
#                     right = len(word)
#                 else:
#                     right -= 1
#     return result # 返回分词结果

# 正向最大匹配算法
def max_match(sen):
    sen = sen.strip()
    # sen = add_spaces_to_numbers(sen)
    sen = sen.split()
    result = []
    for word in sen:
        if trie.search(word):
            result.append(word)
        else:
            left = 0
            right = len(word)
            while left < right:
                if trie.search(word[left:right]):
                    result.append(word[left:right])
                    right = left
                    left = 0
                else:
                    left += 1

    # return result  # 返回分词结果
    return result[::-1]  # 返回分词结果


res_dict = {}

for i in range(len(data_list)):
    sen = data_list[i]
    # sen = remove_chinese_punctuation(sen)
    print(''.join(sen))
    sen = max_match(''.join(sen))
    # 将列表转换为字符串，每个元素之间用空格隔开
    sen = ' '.join(sen)
    print(sen)
    result = transfer_code.transfer(sen)
    # print(result)
    res_dict[i] = result

#将res_dict写入csv文件
import csv

with open('result1.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'expected'])
    for key, value in res_dict.items():
        writer.writerow([key + 1, value])

扬帆远东做与中国合作的先行
扬帆 远东 做 与 中国 合作 的 先行
希腊的经济结构较特殊。
希腊 的 经济 结构 较 特殊 。
海运业雄踞全球之首，按吨位计占世界总数的１７％。
海运 业 雄踞 全球 之 首 ， 按 吨位 计 占 世界 总数 的 １７％ 。
另外旅游、侨汇也是经济收入的重要组成部分，制造业规模相对较小。
另外 旅游 、 侨汇 也是 经济 收入 的 重要 组成部分 ， 制造业 规模 相对 较小 。
多年来，中希贸易始终处于较低的水平，希腊几乎没有在中国投资。
多年来 ， 中 希 贸易 始终 处于 较低 的 水平 ， 希腊 几乎 没有 在 中国 投资 。
十几年来，改革开放的中国经济高速发展，远东在崛起。
十 几年来 ， 改革开放 的 中国经济 高速 发展 ， 远东 在 崛起 。
瓦西里斯的船只中有４０％驶向远东，每个月几乎都有两三条船停靠中国港口。
瓦西 里斯 的 船只 中 有 ４０％ 驶 向 远东 ， 每个 月 几乎 都 有 两 三条 船 停靠 中国 港口 。
他感受到了中国经济发展的大潮。
他 感 受到 了 中国经济 发展 的 大潮 。
他要与中国人合作。
他 要 与 中国人 合作 。
他来到中国，成为第一个访华的大船主。
他 来到 中国 ， 成为 第一个 访 华 的 大船 主 。
访问归来，他对中国发展充满信心，他向希腊海运部长介绍了情况，提出了两国在海运、造船业方面合作的建议。
访问 归来 ， 他 对 中国 发展 充满 信心 ， 他 向 希腊 海运 部长 介绍 了 情况 ， 提出 了 两国 在 海运 、 造船 业 方面 合作 的 建议 。
１９９５年１０月，希腊海运部长访华时，他根据“船长”的建议与中方探讨了在海运、造船方面合作的可能与途径。
１９９５年１０月 ， 希腊 海运 部长 访 华 时 ， 他 根据 “ 船长 ” 的 建议 与 中方 探讨 了 在 海运 、 造船 方面 合作 的 可能 与 途径 。
“船长”本人还与几个船主联合起来准备与我远洋公司建立合资企业。
“ 船长 ” 本人 还 与 几个 船 主 联合 起来 准备 与 我 远洋 公司 建立 合资 企业 。
“船长”常说，要么不干，干就要争第一。
“ 船长 ” 常 说 ， 要么 不干 ， 干 就要 争 第一 。
他拥有世界最大的私人集装箱船队，也要做与中国合作的先行。
他 拥有 世